In [1]:
import pandas as pd
import numpy as np
import itertools
import os

from theano import function
import theano.tensor as tt

from utils.data import load_new_data, train_test_split
from utils.utils import init_weights, morphological_filter
from utils.cv import cv_threshold,cv_fit_model#,cv_build_model
from utils.theano_pq import pq_theano_f, pq_theano
from utils.pass_quality import approx_pq

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '8181' (I am process '9432')
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 4007)
Using Theano backend.


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from utils.utils import model_pq, morphological_filter
from utils.theano_pq import pq_theano

def cv_build_model(X_train):#, metrics=[pq_theano]):
    """
    A function for building Recurrent Neural Network model with
    predefined parameters (X_train used because of input_shape
    parameter estimation)

    :param X_train: training dataset
    :return: model: keras neural netwok model
    """
    model = Sequential()
    model.add(LSTM(input_dim=X_train.shape[2], output_dim=8, return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='tanh'))

    # or with regularized_mse loss
    model.compile(optimizer='rmsprop',
                  loss='MSE',
                #  metrics=metrics
                  )

    return model

# RNN

In [3]:
conc_data = load_new_data()

In [4]:
conc_data['shield_morph'] = morphological_filter(conc_data['r.shield'].values, 3)
conc_data['con_morph'] = morphological_filter(conc_data['r.cor'].values, 3)

In [5]:
used_columns = ['r.cor',
                 #'con_morph',
                 #'r.cor',
                 #'shield_morph',
                 #'r.pass',
                 'y']

X_train, X_test, y_train, y_test = train_test_split(conc_data[used_columns], 
                                                    test_size=0.2,
                                                    n_prev=19)

In [25]:
model = cv_build_model(X_train)

In [26]:
model = cv_fit_model(model, X_train, y_train, nb_epoch=12, weights=True)

Epoch 11/12
Epoch 1/1
54996/54996 [==============================] - 1s - loss: 0.1495     


In [27]:
thres, sm = cv_threshold(X_test, y_test, model, [3])

In [28]:
print thres, sm

0.827203191346 3


In [29]:
y_train_pred = (model.predict(X_train) > thres).astype(int).flatten()

print approx_pq(y_train, morphological_filter(y_train_pred, sm))[:-1]
#print approx_pq(y_train, morphological_filter(y_train_lab, 0))[:-1]

(0.89950980392172253, 5872.0000000099999, 656.0000000000001)


In [30]:
y_test_pred = (model.predict(X_test) > thres).astype(int).flatten()

print approx_pq(y_test, morphological_filter(y_test_pred, sm))[:-1]

#print approx_pq(y_test, morphological_filter(y_test_lab, 0))[:-1]

(0.89454763331399312, 1493.0000000099999, 176.0000000000001)


In [31]:
conc_data[str(used_columns) + str(sm)] = np.concatenate([np.zeros(18), 
                                                         morphological_filter(y_train_pred, sm).astype(int), 
                                                         morphological_filter(y_test_pred, sm).astype(int)])

In [32]:
# approx_pq(conc_data['y'].astype(int), conc_data['r.pass'].astype(int))[:-1]

In [33]:
a, b, c = approx_pq(conc_data['y'].astype(int), conc_data[str(used_columns) + str(sm)].astype(int))[:-1]

(int(a * 100000) / 100000., int(b), int(c))

(0.89828, 7365, 834)

# shield + cor:
(0.88366, 7277, 958)
# only loop:
(0.86061, 6798, 1101);(0.86538, 6840, 1064);(0.87144, 6840, 1009);
# only pass:
(0.8879, 7077, 893);(0.88153, 7017, 943);(0.88531, 7048, 913);(0.88371, 7075, 931);(0.88371, 7075, 931);(0.87849, 7035, 973);
# only shield:
(0.87096, 7209, 1068); (0.8744, 7201, 1034);(0.8909, 7147, 875);(0.89305, 7190, 861);
(0.89437, 7248, 856); $ fantoms and propusk : 88
# only cor:
(0.8734, 7246, 1050); (0.8953, 7269, 850);(0.8988, 7320, 824);(0.8909, 7225, 884);(0.8857, 7136, 920);
(0.9038, 7287, 775);(0.8934, 7255, 865);

In [19]:
approx_pq(conc_data['y'].astype(int), conc_data['y'].astype(int))[:-1]

(1.0, 7873.0000000099999, 1e-13)

In [77]:
approx_pq(conc_data['y'].astype(int), conc_data['r.pass'].astype(int))[:-1]

(0.84931005006734728, 6955.0000000099999, 1234.0)

In [98]:
bench_errors = approx_pq(conc_data['y'].astype(int), conc_data[str(used_columns) + str(sm)].astype(int))[-1][0]

In [99]:
n_fantoms_and_propusk = len([err[1] for err in bench_errors if err[1] == 1])

In [100]:
n_fantoms_and_propusk

242

In [235]:
res_cor = [(0.8953, 7269, 850),
           (0.8988, 7320, 824),
           (0.8909, 7225, 884),
           (0.8857, 7136, 920),
           (0.9038, 7287, 775),
           (0.8934, 7255, 865)]

In [252]:
res_pass = [(0.8879, 7077, 893),
            (0.88153, 7017, 943),
            (0.88531, 7048, 913),
            (0.88371, 7075, 931),
            (0.88371, 7075, 931),
            (0.87849, 7035, 973)]

In [259]:
res_shield = [(0.87096, 7209, 1068),
              (0.8744, 7201, 1034),
              (0.8909, 7147, 875),
              (0.89305, 7190, 861),
              (0.89437, 7248, 856)]

In [254]:
print np.mean([res[0] for res in res_pass])
print np.mean([res[1] for res in res_pass]) * 0.25
print np.mean([res[2] for res in res_pass]) * 0.25

0.883441666667
1763.625
232.666666667


In [260]:
print np.mean([res[0] for res in res_shield])
print np.mean([res[1] for res in res_shield]) * 0.25
print np.mean([res[2] for res in res_shield]) * 0.25

0.884736
1799.75
234.7


In [236]:
np.mean([res[0] for res in res_cor])

0.89464999999999995

In [245]:
np.mean([res[1] for res in res_cor]) * 0.25

1812.1666666666667

In [246]:
np.mean([res[2] for res in res_cor]) * 0.25

213.25

In [251]:
np.array((7277, 958)) * 0.25

array([ 1819.25,   239.5 ])

In [250]:
0.88366

0.88366

In [261]:
7277 * 0.25

1819.25

In [262]:
958 * 0.25

239.5

In [267]:
1819.1 / (239.5 + 1819.1)

0.8836587972408433

In [266]:
6840 / (6840. + 1009)

0.8714485921773474

In [34]:
conc_data

l.cor  l.loop  \
name                                              frame  index                  
M1-69_19_pre_left_20150411125223_pass_sens.ldj    2.0    0          0       0   
                                                  201.0  1          0       1   
                                                  202.0  2          1       1   
                                                  219.0  3          1       1   
                                                  223.0  4          1       1   
                                                  226.0  5          1       1   
                                                  268.0  6          1       1   
                                                  269.0  7          1       0   
                                                  284.0  8          0       0   
                                                  401.0  9          1       0   
                                                  409.0  10         1       0   
                                                  412.0  11         1       0   
                                                  432.0  12         0       0   
                                                  568.0  13         1       0   
                                                  573.0  14         1       1   
                                                  589.0  15         1       1   
                                                  596.0  16         1       1   
                                                  598.0  17         1       1   
                                                  1293.0 18         1       0   
                                                  1301.0 19         1       0   
                                                  1327.0 20         0       0   
                                                  1385.0 21         1       0   
                                                  1389.0 22         1       1   
                                                  1392.0 23         1       1   
                                                  1393.0 24         1       1   
                                                  1399.0 25         1       1   
                                                  1401.0 26         1       1   
                                                  1404.0 27         1       1   
                                                  1470.0 28         1       0   
                                                  1482.0 29         1       0   
...                                                               ...     ...   
M1-177_15_pre_left_20150331024012_pass_sens.ldj   2.0    0          0       0   
                                                  199.0  1          0       0   
                                                  200.0  2          1       0   
                                                  203.0  3          1       1   
                                                  262.0  4          1       1   
                                                  271.0  5          1       0   
                                                  339.0  6          0       0   
                                                  482.0  7          0       0   
M4-339_22_post_left_20150915154007_pass_sens.ldj  2.0    0          0       0   
                                                  189.0  1          0       1   
                                                  200.0  2          0       1   
                                                  201.0  3          1       1   
                                                  231.0  4          1       1   
                                                  232.0  5          1       1   
                                                  395.0  6          1       1   
                                                  397.0  7          0       1   
                                                  413.0  8          1       1   
              